## Neural Network

In [1]:
import torch
import torch.nn as nn #neural net
import torch.nn.functional as F #Functions such as relu
import torch.optim as optim #Parameter optimization functions
from scipy.io import loadmat #Loading data

Define the network

In [2]:
class ANN(nn.Module):
    
    def __init__(self):
        #Define the network
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(400, 25)
        self.fc2 = nn.Linear(25,10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return(x)

### Load the data

In [3]:
def loadData(name):
    return(loadmat(name))

In [4]:
data = loadData("ex3data1.mat")
print("Input shape: ", data["X"].shape)
print("Output shape: ", data["y"].shape)
weights = loadData("ex3weights.mat")
print("Weight Theta1: ", weights["Theta1"].shape)
print("Weight Theta2: ", weights["Theta2"].shape)

Input shape:  (5000, 400)
Output shape:  (5000, 1)
Weight Theta1:  (25, 401)
Weight Theta2:  (10, 26)


### Train the Network

In [5]:
#Initialize the neural net
classifier = ANN()

In [6]:
#Test one input
features = torch.from_numpy(data["X"][0]).float()
features.reshape((1,400))
out = classifier(features)
print(out)

tensor([-0.2001, -0.0027,  0.1366,  0.1569, -0.0891,  0.1674, -0.0935,  0.1087,
         0.2369,  0.0009], grad_fn=<AddBackward0>)


In [7]:
classifier.zero_grad() #Zero the gradients
out.backward(torch.randn(1, 10)) #Backprop with some random values.

### Define the loss function (MSE)

In [8]:
#Define the loss function.
output = classifier(features) #Test on a single image
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.6762, grad_fn=<MseLossBackward>)


In [9]:
#Backprop the loss
classifier.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(classifier.fc1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(classifier.fc1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])
conv1.bias.grad after backward
tensor([-0.1981,  0.0000, -0.1124, -0.0669,  0.0000,  0.0000,  0.0000,  0.1290,
         0.0513,  0.0000,  0.0000,  0.0000,  0.0000, -0.0583,  0.0000, -0.1069,
        -0.0232,  0.0000,  0.0651,  0.0000,  0.0000,  0.0115,  0.0000,  0.0000,
         0.0000])


In [ ]:
#Update the weights.
